In [8]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.io import loadmat

from sklearn import svm

plt.style.use('https://raw.githubusercontent.com/TDAmeritrade/stumpy/main/docs/stumpy.mplstyle')

In [9]:
# Load in the data as a dictionary
Kol=loadmat('Kol_Label.mat')
# Separate keys of the dictionary
KE = Kol['KE']
w_raw = Kol['w']
state = Kol['state']

In [3]:
test_size = 85000
train_size = 10000

state=state.squeeze()
state=state[1:]
[M,N1,N2]=w_raw.shape
w=w_raw.reshape(M,-1)
w=w[1:]
w_train = w[0:train_size,] 
w_test = w[train_size:train_size+test_size,]

In [4]:
model_rbf = svm.OneClassSVM(kernel = 'rbf').fit(w_train)
y_pred_rbf = model_rbf.predict(w_test)


In [5]:
normal_rbf = y_pred_rbf[y_pred_rbf==1]
anomalous_rbf = y_pred_rbf[y_pred_rbf==-1]
print("RBF Kernel Predictions")
print("{} normal, {} anomalous.".format(len(normal_rbf), len(anomalous_rbf)))

RBF Kernel Predictions
44216 normal, 40784 anomalous.


In [6]:
indexes_rbf = []
KE_vals_rbf = []
state_rbf = []

for i in range(train_size, test_size+train_size):
    if y_pred_rbf[i-train_size] == -1:
        indexes_rbf.append(i-train_size)
        KE_vals_rbf.append(Kol['KE'][0][i])
        state_rbf.append(Kol['state'][0][i])


In [7]:
correct_states = state[train_size:train_size+test_size]
correct_states = [-1 if x==1 else x for x in correct_states]
correct_states = [1 if x==0 else x for x in correct_states]

n_correct_rbf = 0
for i in range(0, test_size):
    if correct_states[i] == y_pred_rbf[i]:
        n_correct_rbf += 1
print("Accuracy of the RBF SVM Kernel: {}%".format(n_correct_rbf/test_size*100))

Accuracy of the RBF SVM Kernel: 84.52705882352942%
